<a href="https://colab.research.google.com/github/maleklachheb/Prediction-of-Product-Sales/blob/main/Sale_prediction_MV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)
import missingno
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df = pd.read_csv('/content/drive/MyDrive/dojo/sales_predictions_2023.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [25]:
df.drop_duplicates(inplace=True)
df['Item_Fat_Content'].replace("LF","Low Fat", inplace=True)
df['Item_Fat_Content'].replace("low fat","Low Fat", inplace=True)
df['Item_Fat_Content'].replace("reg","Regular", inplace=True)
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [26]:
# The target we are trying to predict
y = df['Item_Outlet_Sales']
# The features we will use to make the prediction
X = df.drop(columns=['Item_Identifier', 'Item_Outlet_Sales'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [27]:
impute_median = SimpleImputer(strategy='median')
scaler = StandardScaler()
num_pipe = make_pipeline(impute_median, scaler)
col=X_train.select_dtypes("number").columns
num_pipe.fit(X_train[col])


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [28]:
s = ['Medium','High', 'Small','Tier1','Tier2','Tier3','Supermarket Type1','Supermarket Type2','Supermarket Type3','Grocery Store','Low Fat','Regular', 'Tier 1', 'Tier 3', 'Tier 2']
ord_encoder = OrdinalEncoder(categories=s)
scale=StandardScaler()
ord_pipe= make_pipeline(ord_encoder,scale)
ord_cols = ['Outlet_Size','Outlet_Location_Type','Outlet_Type','Item_Fat_Content']

impute_na_ord = SimpleImputer(strategy='most_frequent', fill_value='NA')
ordinal_category_orders = [s,s,s,s]
ord_encoder = OrdinalEncoder(categories=ordinal_category_orders)

ord_pipe = make_pipeline(impute_na_ord, ord_encoder)
ord_pipe.fit(X_train[ord_cols])

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='NA', strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Medium', 'High', 'Small', 'Tier1',
                                             'Tier2', 'Tier3',
                                             'Supermarket Type1',
                                             'Supermarket Type2',
                                             'Supermarket Type3',
                                             'Grocery Store', 'Low Fat',
                                             'Regular', 'Tier 1', 'Tier 3',
                                             'Tier 2'],
                                            ['Medium', 'High', 'Small', 'Tier1',
                                             'Tier2', 'Tier3',
                                             '...
                                             'Regular', 'Tier 1', 'Tier 3',
                                             'Tier 2'],
                                            ['Medium', 'High', 'Small', 'Tier1',
                                             'Tier2', 'Tier3',
                                             'Supermarket Type1',
                                             'Supermarket Type2',
                                             'Supermarket Type3',
                                             'Grocery Store', 'Low Fat',
                                             'Regular', 'Tier 1', 'Tier 3',
                                             'Tier 2'],
                                            ['Medium', 'High', 'Small', 'Tier1',
                                             'Tier2', 'Tier3',
                                             'Supermarket Type1',
                                             'Supermarket Type2',
                                             'Supermarket Type3',
                                             'Grocery Store', 'Low Fat',
                                             'Regular', 'Tier 1', 'Tier 3',
                                             'Tier 2']]))])

In [31]:
ohe_cols =  ['Item_Type']
impute_na = SimpleImputer(strategy='constant', fill_value = "NA")
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Fit the pipeline on the numeric training data
ohe_pipe = make_pipeline(
    impute_na,
    ohe_encoder
)
ohe_pipe.fit(X_train[ohe_cols])


# Transform the training data
X_train_ohe = ohe_pipe.transform(X_train[ohe_cols])
X_test_ohe = ohe_pipe.transform(X_test[ohe_cols])

pd.DataFrame(X_train_ohe).head()

pd.DataFrame(X_test_ohe).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [33]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipe, col),
        ('other', ord_pipe, ord_cols)
    ],
    verbose_feature_names_out=False
)


X_train_processed = preprocessor.fit_transform(X_train)
X_train_processed = preprocessor.transform(X_train)

# Transform the test data
X_test_processed = preprocessor.transform(X_test)

In [34]:
X_train_processed_df = pd.DataFrame(X_train_processed)

# Check the data types of all columns
print("Data Types of Transformed Training Data:")
print(X_train_processed_df.dtypes)

# Check if original numeric features have been scaled
print("\nOriginal Numeric Features After Scaling:")

print(X_train_processed_df[X_train_processed_df.select_dtypes("number").columns].describe())

Data Types of Transformed Training Data:
0    float64
1    float64
2    float64
3    float64
4    float64
5    float64
6    float64
7    float64
dtype: object

Original Numeric Features After Scaling:
                  0             1             2             3            4  \
count  6.392000e+03  6.392000e+03  6.392000e+03  6.392000e+03  6392.000000   
mean  -4.196337e-16 -6.169450e-17  4.668773e-17 -4.250806e-15     0.667240   
std    1.000078e+00  1.000078e+00  1.000078e+00  1.000078e+00     0.884083   
min   -1.969443e+00 -1.291052e+00 -1.767529e+00 -1.532139e+00     0.000000   
25%   -7.968435e-01 -7.624234e-01 -7.638272e-01 -1.293807e+00     0.000000   
50%   -4.988932e-02 -2.318711e-01  3.400912e-02  1.361872e-01     0.000000   
75%    7.682034e-01  5.596016e-01  7.172910e-01  7.320181e-01     2.000000   
max    2.013127e+00  5.132050e+00  1.994559e+00  1.327849e+00     2.000000   

                 5            6            7  
count  6392.000000  6392.000000  6392.000000  
me